In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [ ]:
# Check if CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# CIFAR-10 dataset has 10 classes
num_classes = 10

In [ ]:
# Define the VGG block
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.BatchNorm2d(out_channels))
        layers.append(nn.ReLU(inplace=True))
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

In [ ]:
# Define the VGG network
class VGG(nn.Module):
    def __init__(self, conv_arch, num_classes=10):
        super(VGG, self).__init__()
        self.conv_layers = self._make_layers(conv_arch)
        self.fc_layers = nn.Sequential(
            nn.Linear(conv_arch[-1][1] * 1 * 1, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )

    def _make_layers(self, conv_arch):
        layers = []
        in_channels = 3
        for (num_convs, out_channels) in conv_arch:
            layers.append(vgg_block(num_convs, in_channels, out_channels))
            in_channels = out_channels
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten the features
        x = self.fc_layers(x)
        return x


In [ ]:
# Define the VGG architecture
conv_arch = ((2, 64), (2, 128), (3, 256), (3, 512), (3, 512))


In [ ]:
# Instantiate the model
model = VGG(conv_arch, num_classes=num_classes).to(device)

In [ ]:
# Hyperparameters
learning_rate = 0.001
batch_size = 64
epochs = 10

In [ ]:
# Image preprocessing modules
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [ ]:
# CIFAR-10 data
train_dataset = datasets.CIFAR10(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform)

100%|██████████| 170498071/170498071 [00:05<00:00, 28544958.31it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
# Data loader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Function for training the model
def train(model, criterion, optimizer, train_loader, epochs):
    model.train()
    for epoch in range(epochs):
        for i, (images, labels) in enumerate(train_loader):
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i+1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}')

In [ ]:
# Function for testing the model
def test(model, test_loader):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
        print(f'Accuracy of the model on the test images: {accuracy}%')

In [ ]:
# Train the model
train(model, criterion, optimizer, train_loader, epochs)


Epoch [1/10], Step [100/782], Loss: 2.0801
Epoch [1/10], Step [200/782], Loss: 1.8833
Epoch [1/10], Step [300/782], Loss: 2.0433
Epoch [1/10], Step [400/782], Loss: 1.9912
Epoch [1/10], Step [500/782], Loss: 1.9454
Epoch [1/10], Step [600/782], Loss: 1.8182
Epoch [1/10], Step [700/782], Loss: 1.8854
Epoch [2/10], Step [100/782], Loss: 1.7165
Epoch [2/10], Step [200/782], Loss: 1.5311
Epoch [2/10], Step [300/782], Loss: 1.5388
Epoch [2/10], Step [400/782], Loss: 1.5002
Epoch [2/10], Step [500/782], Loss: 1.4736
Epoch [2/10], Step [600/782], Loss: 1.6082
Epoch [2/10], Step [700/782], Loss: 1.5148
Epoch [3/10], Step [100/782], Loss: 1.2437
Epoch [3/10], Step [200/782], Loss: 1.4022
Epoch [3/10], Step [300/782], Loss: 1.6404
Epoch [3/10], Step [400/782], Loss: 1.2970
Epoch [3/10], Step [500/782], Loss: 1.0881
Epoch [3/10], Step [600/782], Loss: 1.2895
Epoch [3/10], Step [700/782], Loss: 1.2399
Epoch [4/10], Step [100/782], Loss: 1.0608
Epoch [4/10], Step [200/782], Loss: 1.0306
Epoch [4/10

In [ ]:
# Test the model
test(model, test_loader)

Accuracy of the model on the test images: 81.2%
